In [1]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet -O ../data/raw/rides_2022_01.parquet

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [1]:
from pathlib import Path
import requests
import zipfile
import pandas as pd
import pyarrow.parquet as pq

def download_one_file_of_raw_data(year: int) -> Path:
    """"""
    URL = f'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/bicicletas-publicas/recorridos-realizados-{year}.zip'
    response= requests.get(URL) #, stream=True)

    
    if response.status_code == 200:
        # with open(nombre_archivo_zip, 'wb') as file:
        #     for chunk in response.iter_content(chunk_size=8192):  # Tamaño del búfer ajustado a 8192 bytes
        #         file.write(chunk)
        path = f'../data/raw/recorridos-realizados-{year}.zip'
        open(path, "wb").write(response.content)     
        print(f'descargado año {year}')

        return path
    else:
        raise Exception(f'{URL} is not available')

In [2]:
import pyarrow as pa

def unzip_and_convert_csv_to_parquet(year: int) -> Path:
    nombre_archivo_zip = f"../data/raw/recorridos-realizados-{year}.zip"
        # Descomprimir el archivo zip
    with zipfile.ZipFile(nombre_archivo_zip, 'r') as archivo_zip:

        # Extraer el archivo CSV del zip
        nombre_archivo_csv = archivo_zip.namelist()[0]  # Suponiendo que el archivo CSV es el primer archivo en el zip
        archivo_zip.extractall(f"../data/raw/")

        # Leer el archivo CSV con pandas
        df = pd.read_csv(f"../data/raw/{nombre_archivo_csv}", delimiter=',', decimal=".")

        # Convertir el DataFrame a formato parquet
        nombre_archivo_parquet = f"rides_{year}.parquet"
        table = pa.Table.from_pandas(df)
        pq.write_table(table, f"../data/raw/{nombre_archivo_parquet}")



        # table = pq.Table.from_pandas(df)
        # pq.write_table(table, nombre_archivo_parquet)
        #pq.write_table(pq.Table.from_pandas(df), nombre_archivo_parquet)
        path = f'../data/raw/rides_{year}.parquet'
    return path
    

In [8]:
#No sé por qué descarga mal el archivo .zip y tengo que hacerlo manualmente. Dice carpeta no válida.
download_one_file_of_raw_data(year=2022)

KeyboardInterrupt: 

In [9]:
unzip_and_convert_csv_to_parquet(year=2022)

'../data/raw/rides_2022.parquet'

In [10]:
import pandas as pd

rides = pd.read_parquet('../data/raw/rides_2022.parquet')

rides.head(10)

,Unnamed: 0,X,Id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,Género
0,1,1,13267975BAEcobici,"2,610",2022-01-16 14:58:42,5BAEcobici,005 - Plaza Italia,Av. Sarmiento 2601,-58.420954,-34.580550,2022-01-16 15:42:12,210BAEcobici,335 - General Urquiza,Figueroa Alcorta & Sarmiento,-58.411278,-34.572165,776361BAEcobici,ICONIC,FEMALE
1,2,2,13268526BAEcobici,545,2022-01-16 17:26:27,51BAEcobici,051 - TUCUMAN,Tucuman & 9 De Julio Av.,-58.382126,-34.601478,2022-01-16 17:35:32,174BAEcobici,174 - MINISTERIO DE EDUCACION,Marcelo T. de Alvear & Rodriguez Peña,-58.391768,-34.597225,776407BAEcobici,ICONIC,MALE
2,3,3,13268400BAEcobici,"2,061",2022-01-16 16:51:12,161BAEcobici,161 - Humahuaca,3912 Humahuaca,-58.419676,-34.602078,2022-01-16 17:25:33,117BAEcobici,117 - HUMBERTO 1°,Peru 1016,-58.374176,-34.620101,671762BAEcobici,ICONIC,FEMALE
3,4,4,13268164BAEcobici,"12,748",2022-01-16 15:58:01,210BAEcobici,335 - General Urquiza,Figueroa Alcorta & Sarmiento,-58.411278,-34.572165,2022-01-16 19:30:29,382BAEcobici,204 - Biarritz,Biarritz 2403,-58.477390,-34.605470,776361BAEcobici,ICONIC,FEMALE
4,5,5,13270010BAEcobici,"4,337",2022-01-16 23:40:09,215BAEcobici,113 - Guatemala,Guatemala 4773,-58.424996,-34.585878,2022-01-17 00:52:26,205BAEcobici,125 - F.J.Santamaria de Oro,F.J.Santamaria de Oro & Guatemala,-58.428016,-34.583323,454615BAEcobici,ICONIC,OTHER
5,6,6,13269548BAEcobici,"4,243",2022-01-16 20:42:30,268BAEcobici,399 - GARCIA DEL RIO,Av. García del Río 3182,-58.477000,-34.550300,2022-01-16 21:53:13,268BAEcobici,399 - GARCIA DEL RIO,Av. García del Río 3182,-58.477000,-34.550300,200959BAEcobici,ICONIC,OTHER
6,7,7,13268959BAEcobici,932,2022-01-16 18:47:17,278BAEcobici,233 - MONROE,2519 Superi,-58.469813,-34.564122,2022-01-16 19:02:49,236BAEcobici,254 - Plaza Rafael Hernandez,Vuelta de Obligado 2004,-58.455166,-34.562161,823366BAEcobici,ICONIC,FEMALE
7,8,8,13267669BAEcobici,"4,337",2022-01-16 12:55:26,368BAEcobici,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,2022-01-16 14:07:43,368BAEcobici,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,826754BAEcobici,ICONIC,MALE
8,9,9,13267592BAEcobici,"3,735",2022-01-16 12:25:03,382BAEcobici,204 - Biarritz,Biarritz 2403,-58.477390,-34.605470,2022-01-16 13:27:18,5BAEcobici,005 - Plaza Italia,Av. Sarmiento 2601,-58.420954,-34.580550,776361BAEcobici,ICONIC,FEMALE
9,10,10,13376812BAEcobici,366,2022-01-29 15:13:01,433BAEcobici,273 - Plazoleta Colombia,1619 Brandsen,-58.373726,-34.637697,2022-01-29 15:19:07,6BAEcobici,006 - Parque Lezama,"Avenida Martin Garcia, 295",-58.369758,-34.628526,772370BAEcobici,ICONIC,MALE


In [59]:
# rides[rides['nombre_estacion_origen'].str.startswith('147')]

In [58]:
# rides.loc[0:60000,['id_estacion_origen','nombre_estacion_origen']]

In [11]:
rides.shape

(2922805, 19)

In [12]:
rides.id_estacion_origen.nunique()

331

In [12]:
# #Para no realizar un modelo con las 300 estaciones lo haremos para el top 10 en 2022
# viajes_por_estacion = rides['id_estacion_origen'].value_counts()
# top_ten_estaciones = viajes_por_estacion.nlargest(10)
# #print(top_ten_estaciones)
# rides = rides[rides['id_estacion_origen'].isin(top_ten_estaciones.index)]

In [60]:
# rides.nombre_estacion_origen.unique()

In [13]:
#Opción para hacer el ranking de las 10 estaciones. Para no realizar un modelo con las 300 estaciones lo haremos para el top 10 en 2022
# rides_2022 = rides.copy()
# viajes_por_estacion = rides['id_estacion_origen'].value_counts()
# top_ten_estaciones = viajes_por_estacion.nlargest(10)
# rides = rides[rides['id_estacion_origen'].isin(top_ten_estaciones.index)]
# rides = rides.reset_index(drop=True)

#Nos quedamos sólo con las columnas que nos interesan y las renombramos
rides = rides[['fecha_origen_recorrido', 'id_estacion_origen']]
# Eliminar la parte "BAEcobici" y convertir a tipo int
rides['id_estacion_origen'] = rides['id_estacion_origen'].str.replace('BAEcobici', '').astype(int)

rides.rename(columns={
    'fecha_origen_recorrido': 'pickup_datetime',
    'id_estacion_origen': 'pickup_location_id',
}, inplace=True)

rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
rides.head(10)

,pickup_datetime,pickup_location_id
0,2022-01-16 14:58:42,5
1,2022-01-16 17:26:27,51
2,2022-01-16 16:51:12,161
3,2022-01-16 15:58:01,210
4,2022-01-16 23:40:09,215
5,2022-01-16 20:42:30,268
6,2022-01-16 18:47:17,278
7,2022-01-16 12:55:26,368
8,2022-01-16 12:25:03,382
9,2022-01-29 15:13:01,433


In [14]:
rides.pickup_location_id.nunique()

331

In [13]:
# print(top_ten_estaciones)

In [15]:
rides['pickup_datetime'].describe()

C:\Users\jayan\AppData\Local\Temp\ipykernel_13464\3389344848.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 2922805
unique                2689886
top       2022-08-12 14:45:13
freq                        7
first     2022-01-01 00:11:07
last      2022-12-31 23:55:39
Name: pickup_datetime, dtype: object

In [16]:
rides = rides[rides.pickup_datetime >= '2022-01-01']
rides = rides[rides.pickup_datetime < '2023-01-01']
rides['pickup_datetime'].describe()

C:\Users\jayan\AppData\Local\Temp\ipykernel_13464\213125758.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 2922805
unique                2689886
top       2022-08-12 14:45:13
freq                        7
first     2022-01-01 00:11:07
last      2022-12-31 23:55:39
Name: pickup_datetime, dtype: object

In [17]:
rides.to_parquet('../data/transformed/validated_rides_2022.parquet')